# Aula 08 - Aprendendo a Ranquear - Exercícios

In [1]:
import pandas as pd
import numpy as np
import random

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (2).gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv', names=['itemId', 'title', 'genre'], header=0)
ratings = pd.read_csv('./dataset/ratings_sample.csv', names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
df = ratings[['userId', 'itemId', 'rating']]
df = df.merge(movies[['itemId', 'title']])
df

,userId,itemId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


### Mapeamento de ids

In [4]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.itemId.unique())}
df['userId'] = df['userId'].map(map_users)
df['itemId'] = df['itemId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.itemId] = row.title


### Divisão da base em treino e teste

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

## Questão 01

***Exercício 01:*** Comparação entre um algoritmo pair-wise e point-wise:
- Utilizando o BPR implementado em aula, gere uma lista de 10 recomendações para um determinado usuário da base.
- Calcule a precisão média dessas recomendações usando como ground-truth o conjunto de teste. 
- Utilize o algoritmo MatrixFactorization (SVD otimizado) de predição de notas para prever as notas dos itens que aquele usuário não avaliou ainda. 
- Ordene inversamente esses itens pela nota predita, e recomende os 10 primeiros filmes.
- Calcule a precisão média dessas recomendações e compare o resultado com o resultado obtido pelo BPR. 

### Funções

In [6]:
# Obter a lista de todos os itens que um usuário avaliou.
def get_item_ids(df, userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId==userId),'itemId'].tolist())

get_item_ids(df, 0)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [7]:
# Pre-computing observed and unobserved items for each user
observed = dict()
unobserved = dict()
all_users = df['userId'].unique().tolist() # usar conj. total
all_items = df['itemId'].unique().tolist() # usar conj. total

for u in all_users:
    observed[u] = get_item_ids(train, u) # usar conj. de treinamento
    unobserved[u] = list(set(all_items)-set(observed[u]))

In [8]:
def draw(userId):    
    i = random.choice(observed[userId])
    j = random.choice(unobserved[userId])
    return i, j

draw(2)

(28, 32)

In [9]:
def train_bprmf(train, n_factors, lr=0.05, reg=0.02, miter=30, draw=draw):    
    n_users = df['userId'].max()+1
    n_items = df['itemId'].max()+1    
    item_bias = np.zeros(n_items)
    p = np.random.normal(0, 0.1, (n_users, n_factors))
    q = np.random.normal(0, 0.1, (n_items, n_factors))
    
    error = []
    for t in range(miter):
        print('Iter #', t, end='\r')
        sq_error = 0
        random_users = random.choices(train['userId'].unique(), k=len(train))
        for u in random_users:
            i, j = draw(u)
            x_uij = item_bias[i] - item_bias[j] + (np.dot(p[u], q[i]) - np.dot(p[u], q[j]))
            sq_error += x_uij
            
            eps = 1 / (1 + np.exp(x_uij))

            item_bias[i] += lr * (eps - reg * item_bias[i])
            item_bias[j] += lr * (-eps - reg * item_bias[j])            

            # Adjust the factors
            u_f = p[u]
            i_f = q[i]
            j_f = q[j]

            # Compute and apply factor updates
            p[u] += lr * ((i_f - j_f) * eps - reg * u_f)
            q[i] += lr * (u_f * eps - reg * i_f)
            q[j] += lr * (-u_f * eps - reg * j_f)
            
        error.append(sq_error/len(random_users))
            
    return item_bias, p, q, error

In [10]:
def predict(b, p, q, n=10):
    w = b.T + np.dot(p, q.T)
    ranking = []
    
    for u, user in enumerate(all_users):
        partial_ranking = list()
        candidate_items = sorted(range(len(w[u])), key=lambda k: w[u][k], reverse=True)
        
        for i in candidate_items:
            if i not in observed[user]:
                partial_ranking.append((user, i, w[u][i]))

            if len(partial_ranking) == n:
                break

        ranking += partial_ranking
        print('User #', u, end='\r')
        
    return pd.DataFrame(ranking, columns=['userId', 'itemId', 'rating'])

In [11]:
def precision(pred, test, n=10):
    _precisions = dict()
    
    for u in all_users:
        recomended = set(
            pred
            .loc[pred.userId==u]
            .sort_values(by='rating', ascending=False)
            .head(n)
            .itemId
            .tolist()
        )
        
        ground_truth = set(
            test
            .loc[test.userId==u]
            .sort_values(by='rating', ascending=False)
            .head(n)
            .itemId
            .tolist()
        )
        
        if len(ground_truth) == 0:
            _precisions[u] = float('nan')
        else:
            _precisions[u] = len(recomended & ground_truth) / len(ground_truth)
        print(f'Progress: {u}/{len(all_users)}', end='\r')
    
    return pd.DataFrame(list(_precisions.items()), columns=['userId', 'precision'])

### Execução

In [12]:
b, p, q, error = train_bprmf(train, 4)

In [13]:
pred = predict(b, p, q, n=10)
pred.to_csv('precomp/pred.csv', index=False, sep='\t', header=False)
pred.head()

,userId,itemId,rating
0,0,19,2.249112
1,0,17,2.009922
2,0,22,1.944331
3,0,20,1.854991
4,0,18,1.826188


In [14]:
precisions = precision(pred, test, n=10)
precisions.head()

,userId,precision
0,0,0.000000
1,1,0.666667
2,2,1.000000
3,3,0.750000
4,4,0.000000


In [15]:
precisions['precision'].dropna().mean().round(2)

0.51

### Exemplos

In [16]:
# Random user
seed = 1
random.seed(seed)
user = random.choice(all_users)

# Recomend 10 items for the user
u_pred = pred.loc[pred.userId==user].sort_values(by='rating', ascending=False).head(10)
display(u_pred)

# Precision for the user
u_precisions = precisions.loc[precisions.userId==user]
display(u_precisions)

,userId,itemId,rating
22010,2201,12,2.277563
22011,2201,19,2.258675
22012,2201,10,2.032094
22013,2201,17,2.031960
22014,2201,8,2.008327
22015,2201,22,1.956937
22016,2201,20,1.877487
22017,2201,18,1.841502
22018,2201,21,1.815213
22019,2201,14,1.756812


,userId,precision
2201,2201,0.666667


### Fatorando Matrizes

In [17]:
! mkdir -p ./precomp

In [18]:
import itertools as it
from caserec.recommenders.rating_prediction.matrixfactorization import MatrixFactorization

In [19]:
test.to_csv('precomp/test.csv', index=False, sep='\t', header=False)
train.to_csv('precomp/train.csv', index=False, sep='\t', header=False)

In [20]:
zero_pd = pd.DataFrame([(u, i, 0) for u, i in it.product(all_users, all_items)], columns=['userId', 'itemId', 'rating'])
zero_pd.to_csv('precomp/zero.csv', index=False, sep='\t', header=False)

zero_pd.head()

,userId,itemId,rating
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0


In [21]:
# Executing the Matrix Factorization algorithm with optimized svd
MatrixFactorization(
    train_file='precomp/train.csv', 
    test_file='precomp/zero.csv', 
    output_file='precomp/mat_pred.csv', 
    factors=4
).compute()

[Case Recommender: Rating Prediction > Matrix Factorization]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 11090 users and 417 items (4624530 interactions) | sparsity:: 0.00%

training_time:: 27.850868 sec
prediction_time:: 8.080057 sec


Eval:: MAE: 3.378029 RMSE: 3.405992 


In [22]:
mat_pred = pd.read_csv('precomp/mat_pred.csv', sep='\t', names=['userId', 'itemId', 'rating'])
mat_pred.head()

,userId,itemId,rating
0,0,0,3.861757
1,0,1,4.085120
2,0,2,4.274407
3,0,3,4.120717
4,0,4,4.251872


In [23]:
mat_precisions = precision(mat_pred, test, n=10)
mat_precisions.head()

,userId,precision
0,0,0.000000
1,1,0.166667
2,2,0.000000
3,3,0.250000
4,4,0.000000


In [24]:
mat_precisions['precision'].dropna().mean().round(2)

0.16

In [25]:
both_precisions = precisions.merge(mat_precisions, on='userId').rename({'precision_x': 'bprmf_precision', 'precision_y': 'mf_precision'}, axis=1)
both_precisions

,userId,bprmf_precision,mf_precision
0,0,0.000000,0.000000
1,1,0.666667,0.166667
2,2,1.000000,0.000000
3,3,0.750000,0.250000
4,4,0.000000,0.000000
...,...,...,...
11085,11085,0.750000,0.250000
11086,11086,0.500000,0.000000
11087,11087,0.500000,0.250000
11088,11088,1.000000,1.000000


In [26]:
# Recomend 10 items for the user
u_mat_pred = mat_pred.loc[mat_pred.userId==user].sort_values(by='rating', ascending=False).head(10)
display(u_pred)

# Precision for the user
u_mat_precisions = precisions.loc[precisions.userId==user]
display(u_precisions)

,userId,itemId,rating
22010,2201,12,2.277563
22011,2201,19,2.258675
22012,2201,10,2.032094
22013,2201,17,2.031960
22014,2201,8,2.008327
22015,2201,22,1.956937
22016,2201,20,1.877487
22017,2201,18,1.841502
22018,2201,21,1.815213
22019,2201,14,1.756812


,userId,precision
2201,2201,0.666667


Nosso modelo bprmf obteve uma precisão maior quando comparado com a fatoração de matrizes.

## Questão 2

***Exercicio 02:*** Na aula vimos a implementação do método draw() do algoritmo BPR tradicional. Para um dado usuário, este método retorna aleatoriamente um item que ele viu (**item i**) e um outro item que ele não conhece (**item j**). O algoritmo assume que o item visto é preferível ao que ele não viu, e isso é usado para maximizar a diferença entre os scores desses itens (veja a variável **x_uij** da implementação). 

Um problema dessa abordagem é que os itens não vistos são necessariamente encarados como menos relevante pelo algoritmo, o que nem sempre acontece pois pode ser que o usuário não tenha interagido com aquele item pois não o conhece, mas que poderia gostar. Um outro problema é quando ambos os itens i e j foram vistos pelo usuário, o que numa estratégia com feedback exclusivamente implícito o BPR não consegue diferenciar qual item é preferível ao usuário. 

Você consegue pensar numa estratégia aperfeiçoada para o método draw()? Exemplos:
- Se tivermos acesso aos metadados, podemos construir um perfil para o usuário de modo que o método draw() vai retornar itens j que estão mais distantes desse perfil.
- Se dois itens i e j foram vistos, use os metadados (ou as notas, caso use feedback explícito) para decidir qual deles deve ser o i e qual deve ser o j. 
- Etc.

Implemente pelo menos uma estratégia de aperfeiçoamento do BPR, e compare com a versão original. 

In [27]:
def draw2(userId):
    i = random.choice(all_items)
    j = random.choice(all_items)
    
    if i == j: # try again
        return draw2(userId)
    
    elif i in observed[userId] and j in observed[userId]:
        i_rating = train.loc[(train.userId==userId) & (train.itemId==i), 'rating'].values[0]
        j_rating = train.loc[(train.userId==userId) & (train.itemId==j), 'rating'].values[0]
        
        return (i, j) if i_rating > j_rating else (j, i)
        
    elif i in observed[userId]:
        return i, j
    
    elif j in observed[userId]:
        return j, i
    
    
    return i, j

draw2(2)

(291, 410)

In [28]:
b2, p2, q2, error2 = train_bprmf(train, 4, draw=draw2)

In [29]:
pred2 = predict(b2, p2, q2, n=10)
pred2.head()

,userId,itemId,rating
0,0,19,1.293593
1,0,17,0.881169
2,0,18,0.832465
3,0,30,0.789166
4,0,21,0.748637


In [30]:
precisions2 = precision(pred2, test, n=10)
precisions2.head()

,userId,precision
0,0,0.0
1,1,0.5
2,2,1.0
3,3,1.0
4,4,0.0


In [31]:
precisions2['precision'].dropna().mean().round(2)

0.47

In [32]:
# Recomend 10 items for the user
u_pred2 = pred2.loc[pred2.userId==user].sort_values(by='rating', ascending=False).head(10)
display(u_pred2)

# Precision for the user
u_precisions2 = precisions2.loc[precisions2.userId==user]
display(u_precisions2)

,userId,itemId,rating
22010,2201,19,1.295899
22011,2201,12,1.265308
22012,2201,17,0.880854
22013,2201,18,0.835449
22014,2201,30,0.769002
22015,2201,21,0.740742
22016,2201,4,0.671773
22017,2201,22,0.638968
22018,2201,57,0.615082
22019,2201,20,0.606759


,userId,precision
2201,2201,0.5


Nossa função draw2 seleciona aleatoriamente itens x, y e decide qual será o i e j de acordo com os critérios: (1) i está presente em observados; (2) i tem maior nota do que j.

Nosso método customizado obteve uma levemente precisão pior. Provavelmente, a condicional presente na função `draw` está reduzindo a aleatóriedade do sistema e está enviesando o modelo de alguma maneira.